In [ ]:
First make sure you have created an AML environment and jupyter kernel as in the previod bootcamp sessions or from SetUpAMLEnv.ipynb.

Test that you have selected the correct code by running the cell below.

In [ ]:
import emoji
print(emoji.emojize('Python is :thumbs_up:'))

- Create Azure OpenAI and AI Search Resources

- Rename credentials_template.env to credentials.env 

- Fill in the credentials.env with the details of your resources

In [ ]:
# Loads in environement varaibles from credentials.env. 
# Note that if you change the contents of credentials.env you will need to restart your python kernel as well as rerun this cell.
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("credentials.env")

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID")
searchEndpoint = os.environ.get("AZURE_AI_SEARCH_ENDPOINT")
searchIndex = os.environ.get("AZURE_AI_SEARCH_INDEX")
searchKey = os.environ.get("AZURE_AI_SEARCH_API_KEY")

In [ ]:
print("AOAI Endpoint: ", endpoint)
print("AOAI API Key: ", api_key)
print("AOAI Deployment: ", deployment)
print("Search Endpoint: ", searchEndpoint)
print("Search Index: ", searchIndex)
print("Search API Key: ", searchKey)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

article from https://www.bbc.co.uk/sport/articles/cd1qzw9qp8jo

In [ ]:
article = """Formula 1 has tweaked the format to the sprint events for the 2024 season.
Qualifying for the shorter sprint race is now held on Friday instead of Saturday, following the only practice session of the weekend.
The 100km sprint will then take place as the first track action on Saturday, before main qualifying for the grand prix in its regular slot later in the day.
Sunday's race remains unchanged.
China hosted the first sprint of the year in April and Miami the second in May. After Austria this weekend, there will also be sprints in Austin, Brazil and Qatar towards the end of the season.
Why has the sprint format changed again?
The schedule for sprint weekends for this season has been refreshed again in an attempt to create a better spectacle.
In February, teams voted to keep the same sessions, but the order in which they take place has been rejigged.
Last year, Saturday became a standalone sprint day, independent of the main grand prix, with a 'sprint shootout' added to determine who started on pole for the 100km race later that day. Qualifying for Sunday's race was held on Friday, after one hour of practice.
The previous format for sprint weekends, which were introduced in 2021, had the Friday qualifying session setting the grid for the sprint and the result of the sprint establishing the grid for the main grand prix.
That meant drivers were reluctant to take chances in the sprint because any errors or accidents would disproportionately harm their chances in the grand prix.
How does the F1 sprint race work?
The sprint shootout has three sessions - like normal qualifying - with the slowest five cars eliminated at the end of the first two. These sessions last just 12, 10 and eight minutes, respectively. The three sessions in a standard qualifying run for 18, 15 and 12 minutes.
In the sprint race, the top eight finishers score points, from eight for first place down to one for eighth."""

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Consider this article: {article} Please create a summary"}
    ]
)

print(response.choices[0].message.content)

**USE YOUR OWN DATA** - From guide https://learn.microsoft.com/en-us/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID")

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "Summarize the race",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                    "index_name": os.environ["AZURE_AI_SEARCH_INDEX"],
                    "authentication": {
                        "type": "api_key",
                        "key": os.environ["AZURE_AI_SEARCH_API_KEY"],
                    }
                }
            }
        ],   
    },
    temperature=0,
    top_p=1,
    max_tokens=800,
    stop=None,
)

print(completion.model_dump_json(indent=2))

**CODE INTERPRETER**

In [ ]:
import os
import time
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    )

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("Formula1_2022season_raceResults.csv", "rb"),
  purpose='assistants'
)

# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="You are an AI assistant that answers questions on formula 1 race data",
  model=os.environ.get("AZURE_OPENAI_DEPLOYMENT_ID"),
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

In [ ]:
def askCodeInterpreter(prompt):
    # Create a thread
    thread = client.beta.threads.create()

    # Add a user question to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt # Use the prompt from the function argument
    )

    # Run the thread
    run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id
    )

    # Looping until the run completes or fails
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

    if run.status == 'completed':
      messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
      print(messages)
    elif run.status == 'requires_action':
      # the assistant requires calling some functions
      # and submit the tool outputs back to the run
      pass
    else:
      print(run.status)

In [ ]:
askCodeInterpreter(f"Tell me about the dataset")

In [ ]:
askCodeInterpreter(f"which driver received the most points")

In [ ]:
askCodeInterpreter(f"draw me a graph showing the points per race gained by each driver across the seassion so I can see their points accumulation and the performance though the season. Give each driver a different colour")

In [ ]:
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    )

image_data = client.files.content("find the file_id from the response (eg assistant-YgJnoMFC12VsdXRD4fIN0veT)")
image_data_bytes = image_data.read()

with open("./my-image.png", "wb") as file:
    file.write(image_data_bytes)